<a href="https://colab.research.google.com/github/RabaDaba1/pooling-layer-analysis/blob/main/LLaVA_7b_8bit_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content # Cambia el directorio actual a /content, que es donde normalmente se trabaja en Colab.
!git clone -b 5GB https://github.com/camenduru/LLaVA # Clona el repositorio de LLaVA, versión "5GB" (una versión optimizada y más ligera del modelo llava-v1.5-7b).
%cd /content/LLaVA # Entra en la carpeta del repositorio que se acaba de clonar.

!pip install -q transformers==4.36.2 # Instala la versión específica de transformers, necesaria para LLaVA.
!pip install ninja # Instala ninja, una herramienta de compilación rápida usada por PyTorch y otras bibliotecas.
!pip install flash-attn --no-build-isolation # Instala flash-attn, una implementación optimizada de atención para modelos grandes.

!pip install -e . # Instala el propio repositorio LLaVA como un paquete editable.

# !python -m llava.serve.cli \
#     --model-path 4bit/llava-v1.5-7b-5GB \
#     --image-file "https://llava-vl.github.io/static/images/view.jpg" \
#     --load-8bit

In [ ]:
import threading
import subprocess
threading.Thread(target=lambda: subprocess.run(['python3', '-m', 'llava.serve.controller', '--host', '0.0.0.0', '--port', '10000'], check=True), daemon=True).start()
# Inicia el controller, un servidor que coordina las peticiones entre los distintos procesos (modelo, interfaz, etc.).

In [ ]:
import threading
import subprocess
command = [
    'python3', '-m', 'llava.serve.model_worker',
    '--host', '0.0.0.0',
    '--controller', 'http://localhost:10000',
    '--port', '40000',
    '--worker', 'http://localhost:40000',
    '--model-path', '4bit/llava-v1.5-7b-5GB',
    '--load-8bit'
]
threading.Thread(target=lambda: subprocess.run(command, check=True, shell=False), daemon=True).start()
# Lanza el model_worker, que es el servidor que carga y ejecuta el modelo.
# Usa la versión 4bit (cuantizada, ligera) de llava-v1.5-7b-5GB.
# Aunque el path se llama “4bit”, usa --load-8bit, lo cual puede ser una inconsistencia o simplemente una forma de decirle que use cuantización ligera.

In [ ]:
!python3 -m llava.serve.gradio_web_server --controller http://localhost:10000 --model-list-mode reload --share
# Lanza la interfaz Gradio para que puedas interactuar con el modelo desde el navegador.
# --share hace que se cree un enlace público (útil en Colab) para que accedas desde fuera del entorno.